In [1]:
import pandas as pd
import numpy as np

import gurobipy

from pyomo.environ import *
from pyomo.gdp import *

from ortools.constraint_solver import pywrapcp, routing_enums_pb2

In [2]:
def create_data_model():
    
    data = {}
    data['clean_times'] =  [
        [0, 11, 7, 13, 11],
        [5, 0, 13, 15, 15],
        [13, 15, 0, 23, 11],
        [9, 13, 5, 0, 3],
        [3, 7, 7, 7, 0]
    ]
    data['num_machines'] = 1
    data['depot'] = 0
    data['mix_times'] = [40, 35, 45, 32, 50]
    
    return data

In [3]:
data = create_data_model()
manager = pywrapcp.RoutingIndexManager(len(data['clean_times']),
                                       data['num_machines'], data['depot'])
routing = pywrapcp.RoutingModel(manager)

In [4]:
def distance_callback(from_index, to_index):

    from_node = manager.IndexToNode(from_index)
    to_node = manager.IndexToNode(to_index)
    return data['clean_times'][from_node][to_node]

transit_callback_index = routing.RegisterTransitCallback(distance_callback)

In [5]:
routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

search_parameters = pywrapcp.DefaultRoutingSearchParameters()
search_parameters.first_solution_strategy = (
    routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)

In [6]:
def print_solution(manager, routing, solution):

    incompressible_time = np.sum(data['mix_times'])
    print('Objective: {} minutes'.format(solution.ObjectiveValue() + incompressible_time))
    
    index = routing.Start(0)
    plan_output = 'Optimal paint production:\n'
    cleaning_times = 0

    while not routing.IsEnd(index):
        plan_output += ' {} ->'.format(manager.IndexToNode(index)+ 1)
        previous_index = index
        index = solution.Value(routing.NextVar(index)) 
        cleaning_times += routing.GetArcCostForVehicle(previous_index, index, 0)

    plan_output += ' {}\n'.format(manager.IndexToNode(index)+ 1)

    print(plan_output)
    plan_output += 'Optimal paint production: {}minutes\n'.format(cleaning_times)
    

In [7]:
solution = routing.SolveWithParameters(search_parameters)

if solution:
    print_solution(manager, routing, solution)

Objective: 243 minutes
Optimal paint production:
 1 -> 4 -> 3 -> 5 -> 2 -> 1

